In [1]:
import os
import multiprocessing
num_cpus = multiprocessing.cpu_count()

print("Number of CPUs:", num_cpus)

Number of CPUs: 16


In [2]:
from importlib import reload
import matplotlib.pyplot as plt
import mxnet as mx
from mxnet.contrib import amp
import time
from tqdm.notebook import tqdm

import gluoncv as gcv

In [3]:
# Dataset Loading & Transforming
image_size = 480

input_transform_fn = mx.gluon.data.vision.transforms.Compose([
    mx.gluon.data.vision.transforms.Resize(image_size, keep_ratio=True),
    mx.gluon.data.vision.transforms.CenterCrop(image_size),
    mx.gluon.data.vision.transforms.ToTensor(),
    mx.gluon.data.vision.transforms.Normalize([.485, .456, .406], [.229, .224, .225])
])

to_gpu_fn = lambda x: x.as_in_context(mx.gpu())

input_transform_fn_gpu = mx.gluon.data.vision.transforms.Compose([
    to_gpu_fn,
    input_transform_fn
])

# Pre-processing in GPU, with transforms
# then copying back to CPU memory space

# Pre-processing in GPU, with transforms
# Unfortunately, we cannot copy directly into GPU the labels
# Not supported ty ADE20KSegmentation class

to_cpu_fn = lambda x: x.as_in_context(mx.cpu())

input_transform_fn_gpu_cpu = mx.gluon.data.vision.transforms.Compose([
    input_transform_fn_gpu,
    to_cpu_fn
])

# No need for mask transform changes
ade20k_val_cpu = gcv.data.ADE20KSegmentation(split='val')

# Limit to 100 samples
max_samples = 100
samples = range(0, max_samples)

ade20k_val_cpu_pre = mx.gluon.data.SimpleDataset([(ade20k_val_cpu[i][0], ade20k_val_cpu[i][1]) for i in tqdm(samples)])
ade20k_val_gpu_cpu = ade20k_val_cpu_pre.transform_first(input_transform_fn_gpu_cpu, lazy=False)

# Single sample for forward pass (AMP & Quantization requirement)
original_shape = ade20k_val_gpu_cpu[0][0].shape[1:]
single_sample_cpu = ade20k_val_gpu_cpu[0][0].reshape((1, 3) + original_shape)
single_sample_gpu = ade20k_val_gpu_cpu[0][0].reshape((1, 3) + original_shape).as_in_context(mx.gpu())

  0%|          | 0/100 [00:00<?, ?it/s]

[11:16:16] /work/mxnet/src/base.cc:79: cuDNN lib mismatch: linked-against version 8600 != compiled-against version 8204.  Set MXNET_CUDNN_LIB_CHECKING=0 to quiet this warning.


In [4]:
num_workers = 0
batch_size = 4

# DataLoader for data processed in GPU, loaded in CPU
ade20k_val_loader_gpu_cpu = mx.gluon.data.DataLoader(
    ade20k_val_gpu_cpu,
    batch_size=batch_size,
    num_workers=num_workers,
    last_batch="discard")

## Hybridize (CPU)

In [5]:
deeplab_pt_cpu = gcv.model_zoo.get_model('deeplab_resnet101_coco', pretrained=True, ctx=mx.cpu())

start_time = time.time()

for data, _ in tqdm(ade20k_val_loader_gpu_cpu):
    deeplab_pt_cpu(data)

mx.nd.waitall()
print("Time (s):", time.time() - start_time)

  0%|          | 0/25 [00:00<?, ?it/s]

Time (s): 115.22693085670471


In [6]:
deeplab_pt_cpu_hybrid = gcv.model_zoo.get_model('deeplab_resnet101_coco', pretrained=True, ctx=mx.cpu())
deeplab_pt_cpu_hybrid.hybridize()

start_time = time.time()

for data, _ in tqdm(ade20k_val_loader_gpu_cpu):
    deeplab_pt_cpu_hybrid(data)

mx.nd.waitall()
print("Time (s):", time.time() - start_time)

  0%|          | 0/25 [00:00<?, ?it/s]

Time (s): 64.75840330123901


In [7]:
deeplab_pt_cpu_hybrid_mkldnn = gcv.model_zoo.get_model('deeplab_resnet101_coco', pretrained=True, ctx=mx.cpu())
deeplab_pt_cpu_hybrid_mkldnn.hybridize(backend = "MKLDNN")

start_time = time.time()

for data, _ in tqdm(ade20k_val_loader_gpu_cpu):
    deeplab_pt_cpu_hybrid_mkldnn(data)
    
mx.nd.waitall()
print("Time (s):", time.time() - start_time)

  0%|          | 0/25 [00:00<?, ?it/s]

Time (s): 55.860424757003784


In [8]:
deeplab_pt_cpu_hybrid_mkldnn_alloc = gcv.model_zoo.get_model('deeplab_resnet101_coco', pretrained=True, ctx=mx.cpu())
deeplab_pt_cpu_hybrid_mkldnn_alloc.hybridize(backend = "MKLDNN", static_alloc=True)

start_time = time.time()

for data, _ in tqdm(ade20k_val_loader_gpu_cpu):
    deeplab_pt_cpu_hybrid_mkldnn_alloc(data)
    
mx.nd.waitall()
print("Time (s):", time.time() - start_time)

  0%|          | 0/25 [00:00<?, ?it/s]

Time (s): 53.905478715896606


In [10]:
deeplab_pt_cpu_hybrid_mkldnn_alloc_shape = gcv.model_zoo.get_model('deeplab_resnet101_coco', pretrained=True, ctx=mx.cpu())
deeplab_pt_cpu_hybrid_mkldnn_alloc_shape.hybridize(backend = "MKLDNN", static_alloc=True, static_shape=True)

start_time = time.time()

for data, _ in tqdm(ade20k_val_loader_gpu_cpu):
    deeplab_pt_cpu_hybrid_mkldnn_alloc_shape(data)
    
mx.nd.waitall()
print("Time (s):", time.time() - start_time)

  0%|          | 0/25 [00:00<?, ?it/s]

Time (s): 52.464826822280884


## Hybridize (GPU)

In [19]:
deeplab_pt_gpu = gcv.model_zoo.get_model('deeplab_resnet101_coco', pretrained=True, ctx=mx.gpu())

start_time = time.time()

for data, _ in tqdm(ade20k_val_loader_gpu_cpu):
    deeplab_pt_gpu(data.as_in_context(mx.gpu()))
    
mx.nd.waitall()
print("Time (s):", time.time() - start_time)

  0%|          | 0/25 [00:00<?, ?it/s]

Time (s): 13.315197944641113


In [18]:
deeplab_pt_gpu_hybrid = gcv.model_zoo.get_model('deeplab_resnet101_coco', pretrained=True, ctx=mx.gpu())
deeplab_pt_gpu_hybrid.hybridize()

start_time = time.time()

for data, _ in tqdm(ade20k_val_loader_gpu_cpu):
    deeplab_pt_gpu_hybrid(data.as_in_context(mx.gpu()))
    
mx.nd.waitall()
print("Time (s):", time.time() - start_time)

  0%|          | 0/25 [00:00<?, ?it/s]

Time (s): 12.873461246490479


In [22]:
deeplab_pt_gpu_hybrid_static_alloc = gcv.model_zoo.get_model('deeplab_resnet101_coco', pretrained=True, ctx=mx.gpu())
deeplab_pt_gpu_hybrid_static_alloc.hybridize(static_alloc=True)

start_time = time.time()

for data, _ in tqdm(ade20k_val_loader_gpu_cpu):
    deeplab_pt_gpu_hybrid_static_alloc(data.as_in_context(mx.gpu()))
    
mx.nd.waitall()
print("Time (s):", time.time() - start_time)

  0%|          | 0/25 [00:00<?, ?it/s]

Time (s): 12.752988815307617


In [24]:
deeplab_pt_gpu_hybrid_static_alloc_shape = gcv.model_zoo.get_model('deeplab_resnet101_coco', pretrained=True, ctx=mx.gpu())
deeplab_pt_gpu_hybrid_static_alloc_shape.hybridize(static_alloc=True, static_shape=True)

start_time = time.time()

for data, _ in tqdm(ade20k_val_loader_gpu_cpu):
    deeplab_pt_gpu_hybrid_static_alloc_shape(data.as_in_context(mx.gpu()))
    
mx.nd.waitall()
print("Time (s):", time.time() - start_time)

  0%|          | 0/25 [00:00<?, ?it/s]

Time (s): 12.583650827407837


## Automatic Mixed Precision (AMP)

In [25]:
from mxnet.contrib import amp

### AMP CPU - Only with MKLDNN

In [40]:
deeplab_pt_cpu_hybrid = gcv.model_zoo.get_model('deeplab_resnet101_coco', pretrained=True, ctx=mx.cpu())
deeplab_pt_cpu_hybrid.hybridize(backend="MKLDNN", static_alloc=True, static_shape=True)
deeplab_pt_cpu_hybrid(single_sample_cpu);

In [41]:
deeplab_pt_cpu_hybrid_amp = amp.convert_hybrid_block(deeplab_pt_cpu_hybrid, ctx=mx.cpu())

In [42]:
start_time = time.time()

for data, _ in tqdm(ade20k_val_loader_gpu_cpu):
    deeplab_pt_cpu_hybrid_amp(data)
    
mx.nd.waitall()
print("Time (s):", time.time() - start_time)

  0%|          | 0/25 [00:00<?, ?it/s]

Time (s): 56.16465926170349


### AMP GPU

In [50]:
deeplab_pt_gpu_hybrid = gcv.model_zoo.get_model('deeplab_resnet101_coco', pretrained=True, ctx=mx.gpu())
deeplab_pt_gpu_hybrid.hybridize(static_alloc=True, static_shape=True)
deeplab_pt_gpu_hybrid(single_sample_gpu);

In [51]:
deeplab_pt_gpu_hybrid_amp = amp.convert_hybrid_block(deeplab_pt_gpu_hybrid, ctx=mx.gpu())

In [53]:
start_time = time.time()

for data, _ in tqdm(ade20k_val_loader_gpu_cpu):
    deeplab_pt_gpu_hybrid_amp(data.as_in_context(mx.gpu()))
    
mx.nd.waitall()
print("Time (s):", time.time() - start_time)

  0%|          | 0/25 [00:00<?, ?it/s]

Time (s): 3.431182861328125


## MXNet Profiler

### CPU Profiling

In [5]:
deeplab_pt_cpu = gcv.model_zoo.get_model('deeplab_resnet101_coco', pretrained=True, ctx=mx.cpu())

In [6]:
mx.profiler.set_config(
    profile_all=True,
    aggregate_stats=True,
    continuous_dump=True,
    filename='profile_output_cpu.json')

In [7]:
mx.profiler.set_state('run')

deeplab_pt_cpu(single_sample_cpu)

# Wait until all operations have completed
mx.nd.waitall()
# Stop recording
mx.profiler.set_state('stop')
# Log results
mx.profiler.dump()

In [5]:
deeplab_pt_cpu_hybrid = gcv.model_zoo.get_model('deeplab_resnet101_coco', pretrained=True, ctx=mx.cpu())
deeplab_pt_cpu_hybrid.hybridize(backend="MKLDNN", static_alloc=True, static_shape=True)

In [6]:
mx.profiler.set_config(
    profile_all=True,
    aggregate_stats=True,
    continuous_dump=True,
    filename='profile_output_cpu_hybrid.json')

In [7]:
mx.profiler.set_state('run')

deeplab_pt_cpu_hybrid(single_sample_cpu)

# Wait until all operations have completed
mx.nd.waitall()
# Stop recording
mx.profiler.set_state('stop')
# Log results
mx.profiler.dump()

### GPU Profiling

In [5]:
deeplab_pt_gpu = gcv.model_zoo.get_model('deeplab_resnet101_coco', pretrained=True, ctx=mx.gpu())

In [7]:
mx.profiler.set_config(
    profile_all=True,
    aggregate_stats=True,
    continuous_dump=True,
    filename='profile_output_gpu.json')

In [8]:
mx.profiler.set_state('run')

deeplab_pt_gpu(single_sample_gpu)

# Wait until all operations have completed
mx.nd.waitall()
# Stop recording
mx.profiler.set_state('stop')
# Log results
mx.profiler.dump()

In [5]:
deeplab_pt_gpu_hybrid = gcv.model_zoo.get_model('deeplab_resnet101_coco', pretrained=True, ctx=mx.gpu())
deeplab_pt_gpu_hybrid.hybridize(static_alloc=True, static_shape=True)

In [6]:
mx.profiler.set_config(
    profile_all=True,
    aggregate_stats=True,
    continuous_dump=True,
    filename='profile_output_gpu_hybrid.json')

In [7]:
mx.profiler.set_state('run')

deeplab_pt_gpu_hybrid(single_sample_gpu)

# Wait until all operations have completed
mx.nd.waitall()
# Stop recording
mx.profiler.set_state('stop')
# Log results
mx.profiler.dump()

[11:00:02] /work/mxnet/src/operator/nn/./cudnn/./cudnn_algoreg-inl.h:96: Running performance tests to find the best convolution algorithm, this can take a while... (set the environment variable MXNET_CUDNN_AUTOTUNE_DEFAULT to 0 to disable)


## Quantisation (INT8) - CPU Only

In [54]:
from mxnet.contrib import quantization

### Calibration Dataset

In [55]:
# Dataset Loading & Transforming
# Limit to 10 samples (last ones)
max_samples = 10
samples = range(0, max_samples)

ade20k_cal_cpu_pre = mx.gluon.data.SimpleDataset([(ade20k_val_cpu[-i][0], ade20k_val_cpu[-i][1]) for i in tqdm(samples)])
ade20k_cal_gpu     = ade20k_cal_cpu_pre.transform_first(input_transform_fn_gpu, lazy=False)
ade20k_cal_gpu_cpu = ade20k_cal_gpu.transform_first(to_cpu_fn, lazy=False)

  0%|          | 0/10 [00:00<?, ?it/s]

In [56]:
# DataLoader for Calibration
# For CPU, Pre-processed in GPU, copied back to CPU memory space)
num_workers = 0
batch_size = 4

# DataLoader for calibration data processed in GPU, loaded in CPU
ade20k_cal_loader_gpu_cpu = mx.gluon.data.DataLoader(
    ade20k_cal_gpu_cpu,
    batch_size=batch_size,
    num_workers=num_workers,
    last_batch="discard")

In [61]:
deeplab_pt_cpu = gcv.model_zoo.get_model('deeplab_resnet101_coco', pretrained=True, ctx=mx.cpu())
deeplab_pt_cpu(single_sample_cpu);

In [62]:
# Log Quantization info
import logging
logging.basicConfig()
logger = logging.getLogger('logger')
logger.setLevel(logging.INFO)

In [63]:
deeplab_pt_cpu_q_hybrid = quantization.quantize_net_v2(
    deeplab_pt_cpu,
    quantized_dtype='auto',
    exclude_layers=None,
    exclude_layers_match=None,
    calib_data=ade20k_cal_loader_gpu_cpu,
    calib_mode='entropy',
    logger=logger,
    ctx=mx.cpu())

INFO:logger:Export HybridBlock
INFO:logger:These layers have been excluded []
INFO:logger:Quantizing graph
[11:43:59] /work/mxnet/src/operator/quantization/quantize_graph_pass.cc:302: sg_mkldnn_conv_bn_act_0 is quantized.
[11:43:59] /work/mxnet/src/operator/quantization/quantize_graph_pass.cc:302: sg_mkldnn_conv_bn_act_1 is quantized.
[11:43:59] /work/mxnet/src/operator/quantization/quantize_graph_pass.cc:302: sg_mkldnn_conv_bn_act_2 is quantized.
[11:43:59] /work/mxnet/src/operator/quantization/quantize_graph_pass.cc:302: deeplabv327_resnetv1s_pool0_fwd is quantized.
[11:43:59] /work/mxnet/src/operator/quantization/quantize_graph_pass.cc:302: sg_mkldnn_conv_bn_act_3 is quantized.
[11:43:59] /work/mxnet/src/operator/quantization/quantize_graph_pass.cc:302: sg_mkldnn_conv_bn_act_4 is quantized.
[11:43:59] /work/mxnet/src/operator/quantization/quantize_graph_pass.cc:302: sg_mkldnn_conv_bn_6 is quantized.
[11:43:59] /work/mxnet/src/operator/quantization/quantize_graph_pass.cc:302: sg_mkld

In [64]:
deeplab_pt_cpu_q_hybrid.hybridize(backend="MKLDNN", static_alloc=True, static_shape=True)

In [65]:
start_time = time.time()

for data, _ in tqdm(ade20k_val_loader_gpu_cpu):
    deeplab_pt_cpu_q_hybrid(data)
    
mx.nd.waitall()
print("Time (s):", time.time() - start_time)

  0%|          | 0/25 [00:00<?, ?it/s]

Time (s): 36.10324692726135


## ONNX Export

In [1]:
# ONNX library must be installed for this
# !python3 -m pip install onnx
onnx_file_name = "deeplab_resnet101_coco_pt_gpu_hybrid.onnx"

In [67]:
deeplab_pt_gpu_hybrid = gcv.model_zoo.get_model('deeplab_resnet101_coco', pretrained=True, ctx=mx.gpu())
deeplab_pt_gpu_hybrid.hybridize(static_alloc=True, static_shape=True)
deeplab_pt_gpu_hybrid(single_sample_gpu)

# Need to be exported externally for the symbols to be loaded
deeplab_pt_gpu_hybrid_filename = "deeplab_resnet101_coco_pt_gpu_hybrid"
deeplab_pt_gpu_hybrid.export(deeplab_pt_gpu_hybrid_filename)

# Files exported
sym_filename = deeplab_pt_gpu_hybrid_filename + "-symbol.json"
params_filename = deeplab_pt_gpu_hybrid_filename + "-0000.params"

# Verify generated files
assert os.path.exists(sym_filename)
assert os.path.exists(params_filename)

In [68]:
in_shapes = [single_sample_gpu.shape]
in_types = [mx.np.float32]

onnx_model_path = mx.onnx.export_model(
    sym_filename,
    params_filename,
    in_shapes,
    in_types,
    onnx_file_name)

onnx_model_path

'deeplab_resnet101_coco_pt_gpu_hybrid.onnx'

In [69]:
# Model Verification
import onnx

# Load the ONNX model
onnx_model = onnx.load_model(onnx_model_path)

# Check the ONNX graph
onnx.checker.check_graph(onnx_model.graph)

## TensorRT Export

In [2]:
import tensorrt as trt

In [3]:
trt_file_name = "deeplab_resnet101_coco_pt_gpu_hybrid.trt"

In [4]:
TRT_LOGGER = trt.Logger(trt.Logger.INFO)
builder = trt.Builder(TRT_LOGGER)
config = builder.create_builder_config()

explicit_batch = 1 << (int) (trt.NetworkDefinitionCreationFlag.EXPLICIT_BATCH)
deeplab_pt_gpu_hybrid_trt = builder.create_network(explicit_batch)

[10/21/2023-11:59:27] [TRT] [I] [MemUsageChange] Init CUDA: CPU +307, GPU +0, now: CPU 331, GPU 338 (MiB)
[10/21/2023-11:59:30] [TRT] [I] [MemUsageChange] Init builder kernel library: CPU +262, GPU +74, now: CPU 647, GPU 412 (MiB)
[10/21/2023-11:59:30] [TRT] [W] CUDA lazy loading is not enabled. Enabling it can significantly reduce device memory usage. See `CUDA_MODULE_LOADING` in https://docs.nvidia.com/cuda/cuda-c-programming-guide/index.html#env-vars


In [5]:
with open(onnx_file_name, 'rb') as model:
    with trt.OnnxParser(deeplab_pt_gpu_hybrid_trt, TRT_LOGGER) as parser:
        assert parser.parse(model.read()) == True
    deeplab_pt_gpu_hybrid_engine_serialized = builder.build_serialized_network(deeplab_pt_gpu_hybrid_trt, config=config)

[10/21/2023-11:59:34] [TRT] [W] onnx2trt_utils.cpp:377: Your ONNX model has been generated with INT64 weights, while TensorRT does not natively support INT64. Attempting to cast down to INT32.
[10/21/2023-11:59:37] [TRT] [I] [MemUsageChange] Init cuBLAS/cuBLASLt: CPU +496, GPU +214, now: CPU 1612, GPU 626 (MiB)
[10/21/2023-11:59:37] [TRT] [I] [MemUsageChange] Init cuDNN: CPU +114, GPU +52, now: CPU 1726, GPU 678 (MiB)
[10/21/2023-11:59:37] [TRT] [I] Local timing cache in use. Profiling results in this builder pass will not be stored.
[10/21/2023-12:00:16] [TRT] [I] Total Activation Memory: 16775153152
[10/21/2023-12:00:16] [TRT] [I] Detected 1 inputs and 2 output network tensors.
[10/21/2023-12:00:16] [TRT] [I] Total Host Persistent Memory: 342288
[10/21/2023-12:00:16] [TRT] [I] Total Device Persistent Memory: 3019776
[10/21/2023-12:00:16] [TRT] [I] Total Scratch Memory: 134217728
[10/21/2023-12:00:16] [TRT] [I] [MemUsageStats] Peak memory usage of TRT CPU/GPU memory allocators: CPU 25

In [6]:
with open(trt_file_name, 'wb') as f:
    f.write(bytearray(deeplab_pt_gpu_hybrid_engine_serialized))

In [7]:
# Check it can be read back
runtime = trt.Runtime(TRT_LOGGER)

with open(trt_file_name, 'rb') as f:
    deeplab_pt_gpu_hybrid_engine_deserialized = runtime.deserialize_cuda_engine(f.read())

[10/21/2023-12:00:39] [TRT] [I] Loaded engine size: 246 MiB
[10/21/2023-12:00:39] [TRT] [I] [MemUsageChange] Init cuDNN: CPU +0, GPU +10, now: CPU 2433, GPU 1108 (MiB)
[10/21/2023-12:00:39] [TRT] [I] [MemUsageChange] TensorRT-managed allocation in engine deserialization: CPU +0, GPU +248, now: CPU 0, GPU 248 (MiB)
